<p style="display:inline-block;position:relative;font-size:30px;background-color: #d9edab;padding:200px 0px;width:100%;text-align:center">ALL LEANDING CLUB</p><br><strong style="display:block;font-size:30px;padding-top:20px">GA</strong>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time as time

!pip install scikit-learn-intelex

from sklearnex import patch_sklearn
patch_sklearn()

import warnings
warnings.filterwarnings('ignore')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 87 kB 5.7 MB/s 
     |████████████████████████████████| 26.0 MB 1.6 MB/s 
     |████████████████████████████████| 300.3 MB 14 kB/s 
     |████████████████████████████████| 4.0 MB 41.1 MB/s 


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
data = pd.read_csv("https://raw.githubusercontent.com/hsuecu/data_set/main/loan_data.csv")
data.head()

,credit.policy,purpose,int.rate,installment,log.annual.inc,dti,fico,days.with.cr.line,revol.bal,revol.util,inq.last.6mths,delinq.2yrs,pub.rec,not.fully.paid
0,1,debt_consolidation,0.1189,829.10,11.350407,19.48,737,5639.958333,28854,52.1,0,0,0,0
1,1,credit_card,0.1071,228.22,11.082143,14.29,707,2760.000000,33623,76.7,0,0,0,0
2,1,debt_consolidation,0.1357,366.86,10.373491,11.63,682,4710.000000,3511,25.6,1,0,0,0
3,1,debt_consolidation,0.1008,162.34,11.350407,8.10,712,2699.958333,33667,73.2,1,0,0,0
4,1,credit_card,0.1426,102.92,11.299732,14.97,667,4066.000000,4740,39.5,0,1,0,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9578 entries, 0 to 9577
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   credit.policy      9578 non-null   int64  
 1   purpose            9578 non-null   object 
 2   int.rate           9578 non-null   float64
 3   installment        9578 non-null   float64
 4   log.annual.inc     9578 non-null   float64
 5   dti                9578 non-null   float64
 6   fico               9578 non-null   int64  
 7   days.with.cr.line  9578 non-null   float64
 8   revol.bal          9578 non-null   int64  
 9   revol.util         9578 non-null   float64
 10  inq.last.6mths     9578 non-null   int64  
 11  delinq.2yrs        9578 non-null   int64  
 12  pub.rec            9578 non-null   int64  
 13  not.fully.paid     9578 non-null   int64  
dtypes: float64(6), int64(7), object(1)
memory usage: 1.0+ MB


In [4]:
data = pd.get_dummies(data, columns=['purpose'], drop_first=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9578 entries, 0 to 9577
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   credit.policy               9578 non-null   int64  
 1   int.rate                    9578 non-null   float64
 2   installment                 9578 non-null   float64
 3   log.annual.inc              9578 non-null   float64
 4   dti                         9578 non-null   float64
 5   fico                        9578 non-null   int64  
 6   days.with.cr.line           9578 non-null   float64
 7   revol.bal                   9578 non-null   int64  
 8   revol.util                  9578 non-null   float64
 9   inq.last.6mths              9578 non-null   int64  
 10  delinq.2yrs                 9578 non-null   int64  
 11  pub.rec                     9578 non-null   int64  
 12  not.fully.paid              9578 non-null   int64  
 13  purpose_credit_card         9578 

In [5]:
X = data.drop(['not.fully.paid'], axis=1)
y = data['not.fully.paid']

In [6]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
col=['days.with.cr.line', 'revol.bal']
X[col] = scaler.fit_transform(X[col])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, train_size=0.7)
                     
                     
print(f'X_train:{X_train.shape}\n\
X_test:{X_test.shape}\n\
y_train:{y_train.shape}\n\
y_test:{y_test.shape}')

X_train:(6704, 18)
X_test:(2874, 18)
y_train:(6704,)
y_test:(2874,)


In [7]:
# function to evaluate
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score, f1_score, average_precision_score
from sklearn.model_selection import StratifiedKFold
def analyse(model, xtrain, ytrain, xtest, ytest):
    ytrain_pred = model.predict(xtrain)
    ytest_pred = model.predict(xtest)
    try:
        y_score_test = model.decision_function(xtest)
        y_score_train = model.decision_function(xtrain)
    except:
        y_score_test = model.predict_proba(xtest)[:,1]
        y_score_train = model.predict_proba(xtrain)[:,1]

    ret = {'train':{'pr auc': average_precision_score(ytrain, y_score_train),
                'accuracy': accuracy_score(ytrain, ytrain_pred),
                'recall':recall_score(ytrain, ytrain_pred),
                'precision':precision_score(ytrain, ytrain_pred),
                'roc auc':roc_auc_score(ytrain, y_score_train),
                'f1 score': f1_score(ytrain, ytrain_pred)},
           'test':{'pr auc': average_precision_score(ytest, y_score_test),
                'accuracy':accuracy_score(ytest, ytest_pred),
                'recall':recall_score(ytest, ytest_pred),
                'precision':precision_score(ytest, ytest_pred),
                'roc auc':roc_auc_score(ytest, y_score_test),
                'f1 score': f1_score(ytest, ytest_pred)}}
    return ret

In [8]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score, f1_score, auc, precision_recall_curve

def analyseANN(model, xtrain, ytrain, xtest, ytest):
    ypred =  model.predict(xtrain)
    ypred_test = model.predict(xtest)
    precision_train, recall_train, thresholds = precision_recall_curve(ytrain, ypred)
    precision_test, recall_test, thresholds = precision_recall_curve(y_test, ypred_test)

    for i in range(len(ypred)):
        if ypred[i] <=0.5:
            ypred[i] =0
        else: 
            ypred[i] =1
    for i in range(len(ypred_test)):
        if ypred_test[i] <=0.5:
            ypred_test[i] =0
        else: 
            ypred_test[i] =1
    accuracy = accuracy_score(ytrain, ypred)
    recall = recall_score(ytrain, ypred)
    precision = precision_score(ytrain, ypred)
    roc_auc = roc_auc_score(ytrain, ypred)
    f1score = f1_score(ytrain, ypred)

    ret = {'train':{'pr auc': auc(recall_train, precision_train),
                    'accuracy': accuracy,
                    'recall':recall,
                    'precision':precision,
                    'roc auc':roc_auc,
                    'f1 score': f1score},
               'test':{'pr auc': auc(recall_test, precision_test),
                    'accuracy':accuracy_score(ytest, ypred_test),
                    'recall':recall_score(ytest, ypred_test),
                    'precision':precision_score(ytest, ypred_test),
                    'roc auc':roc_auc_score(ytest, ypred_test),
                    'f1 score': f1_score(ytest, ypred_test)}}
    return ret

In [9]:
##                     ##
## LOGISTIC REGRESSION ##
##                     ##

from sklearn.linear_model import LogisticRegression
import json
lr = LogisticRegression(random_state=42)
lr.fit(X_train, y_train)
print(json.dumps(analyse(lr, X_train, y_train, X_test, y_test), indent=2))

{
  "train": {
    "pr auc": 0.2786964082880774,
    "accuracy": 0.836664677804296,
    "recall": 0.01646843549862763,
    "precision": 0.47368421052631576,
    "roc auc": 0.6622082522192472,
    "f1 score": 0.031830238726790444
  },
  "test": {
    "pr auc": 0.26791885489569,
    "accuracy": 0.8469032707028532,
    "recall": 0.011363636363636364,
    "precision": 0.5,
    "roc auc": 0.6697803839545828,
    "f1 score": 0.022222222222222223
  }
}


In [10]:
##                     ##
## XG BOOST CLASSIFIER ##
##                     ##

from xgboost.sklearn import XGBClassifier
params = {
            'booster':'gbtree',
            'objective':'binary:logistic',
            'max_depth': 3,
            'learning_rate': 1,
            'n_estimators':100,
            "eval_metric": ['logloss'],
            'random_state': 42
        }         
           
          
# instantiate the classifier 
xgb_clf = XGBClassifier(**params)
xgb_clf.fit(X_train, y_train)
print(json.dumps(analyse(xgb_clf, X_train, y_train, X_test, y_test), indent=2))

{
  "train": {
    "pr auc": 0.8350199033736918,
    "accuracy": 0.9160202863961814,
    "recall": 0.5288197621225984,
    "precision": 0.9233226837060703,
    "roc auc": 0.9437361880491252,
    "f1 score": 0.6724840023269343
  },
  "test": {
    "pr auc": 0.20988272404206854,
    "accuracy": 0.8016701461377871,
    "recall": 0.1590909090909091,
    "precision": 0.25925925925925924,
    "roc auc": 0.5981651975797415,
    "f1 score": 0.19718309859154928
  }
}


In [11]:
##                          ##
## RANDOM FOREST CLASSIFIER ##
##                          ##


from sklearn.ensemble import RandomForestClassifier
rforest = RandomForestClassifier(max_depth = 3)
rforest.fit(X_train, y_train)
print(json.dumps(analyse(rforest, X_train, y_train, X_test, y_test), indent=2))

{
  "train": {
    "pr auc": 0.32916918269085393,
    "accuracy": 0.8369630071599046,
    "recall": 0.0,
    "precision": 0.0,
    "roc auc": 0.6904537274269941,
    "f1 score": 0.0
  },
  "test": {
    "pr auc": 0.2637890920828436,
    "accuracy": 0.8469032707028532,
    "recall": 0.0,
    "precision": 0.0,
    "roc auc": 0.6741022073653544,
    "f1 score": 0.0
  }
}


In [12]:
##                        ##
## NAIVE BAYES CLASSIFIER ##
##                        ##


from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)
print(json.dumps(analyse(gnb, X_train, y_train, X_test, y_test), indent=2))

{
  "train": {
    "pr auc": 0.26415148108775743,
    "accuracy": 0.7699880668257757,
    "recall": 0.2991765782250686,
    "precision": 0.2964641885766092,
    "roc auc": 0.6571487551491377,
    "f1 score": 0.29781420765027317
  },
  "test": {
    "pr auc": 0.2589611520282426,
    "accuracy": 0.7825330549756437,
    "recall": 0.3181818181818182,
    "precision": 0.3010752688172043,
    "roc auc": 0.6769169716889519,
    "f1 score": 0.30939226519337015
  }
}


In [13]:
##                ##
## SVC CLASSIFIER ##
##                ##


from sklearn.svm import SVC
svc = SVC(C=1, kernel='rbf', gamma='scale')
svc.fit(X_train, y_train)
print(json.dumps(analyse(svc, X_train, y_train, X_test, y_test), indent=2))

{
  "train": {
    "pr auc": 0.22652523785324272,
    "accuracy": 0.8369630071599046,
    "recall": 0.0,
    "precision": 0.0,
    "roc auc": 0.584732838368236,
    "f1 score": 0.0
  },
  "test": {
    "pr auc": 0.19761800002986032,
    "accuracy": 0.8469032707028532,
    "recall": 0.0,
    "precision": 0.0,
    "roc auc": 0.5803475386569059,
    "f1 score": 0.0
  }
}


In [14]:
##                ##
## ANN CLASSIFIER ##
##                ##


import sklearn
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor

# Import necessary modules
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import r2_score

def Model():
    ann = MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=10, activation='relu',solver='adam')
    return ann

In [15]:
_start = time.time()

model= Model()

model.fit(X_train, y_train)

ret = analyseANN(model, X_train, y_train, X_test, y_test)

_end = time.time()

print(f'running for: {int((_end-_start)/60)}m, {int((_end-_start)%60)}s')
print(json.dumps(ret, indent=2))

running for: 0m, 0s
{
  "train": {
    "pr auc": 0.36112774539241765,
    "accuracy": 0.7173329355608592,
    "recall": 0.3687099725526075,
    "precision": 0.2506218905472637,
    "roc auc": 0.5769766223483052,
    "f1 score": 0.29840799703813403
  },
  "test": {
    "pr auc": 0.35137648317962,
    "accuracy": 0.7143354210160056,
    "recall": 0.375,
    "precision": 0.2320675105485232,
    "roc auc": 0.5753389482333607,
    "f1 score": 0.2867072111207645
  }
}


# GA OPTIMIZED

In [16]:
!pip install sklearn-genetic-opt


from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits
from sklearn.metrics import accuracy_score
from sklearn_genetic import GASearchCV
from sklearn_genetic.space import Continuous, Categorical, Integer
from sklearn_genetic.plots import plot_fitness_evolution, plot_search_space
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 160 kB 25.9 MB/s 


In [17]:
##                     ##
## LOGISTIC REGRESSION ##
##                     ##

from sklearn.linear_model import LogisticRegression
import json

param_grid = {
    'max_iter': Integer(100, 1000),
    'penalty': Categorical(['l1', 'l2']),
    'C': Continuous(1e-6, 1e-2, distribution="log-uniform"),
    'l1_ratio': Continuous(1e-9, 1, distribution='log-uniform')
}


lr = LogisticRegression(solver="saga")
cv = StratifiedKFold(n_splits=3, shuffle=True)

evolved_estimator = GASearchCV(estimator=lr, cv=cv,
                               scoring='accuracy', population_size=10,
                               generations=10, tournament_size=3,
                               elitism=True, crossover_probability=0.8,
                               mutation_probability=0.1, param_grid=param_grid,
                               criteria='max', algorithm='eaMuPlusLambda',
                               n_jobs=-1, verbose=True, keep_top_k=4)

In [18]:
_start =time.time()

evolved_estimator.fit(X_train, y_train)

_end = time.time()

# only in seconds

print(f'running for: {int((_end-_start)/60)}m, {int((_end-_start)%60)}s')

gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	10    	0.836933	5.96805e-05	0.836963   	0.836814   
1  	18    	0.836978	4.47408e-05	0.837112   	0.836963   
2  	19    	0.836978	4.47474e-05	0.837112   	0.836963   
3  	18    	0.836978	4.47338e-05	0.837112   	0.836963   
4  	19    	0.837008	9.54759e-05	0.837112   	0.836814   
5  	17    	0.837038	7.45579e-05	0.837112   	0.836963   
6  	17    	0.837082	5.96436e-05	0.837112   	0.836963   
7  	18    	0.837097	4.47138e-05	0.837112   	0.836963   
8  	18    	0.837112	2.67041e-08	0.837112   	0.837112   
9  	16    	0.837097	4.47605e-05	0.837112   	0.836963   
10 	19    	0.837097	4.4745e-05 	0.837112   	0.836963   
running for: 4m, 2s


In [19]:
print(json.dumps(analyse(evolved_estimator, X_train, y_train, X_test, y_test), indent=2))

print(evolved_estimator.best_params_)

{
  "train": {
    "pr auc": 0.25080797681316497,
    "accuracy": 0.8369630071599046,
    "recall": 0.0018298261665141812,
    "precision": 0.5,
    "roc auc": 0.6345876931390324,
    "f1 score": 0.003646308113035552
  },
  "test": {
    "pr auc": 0.2442650844379986,
    "accuracy": 0.8469032707028532,
    "recall": 0.0022727272727272726,
    "precision": 0.5,
    "roc auc": 0.6420613281541794,
    "f1 score": 0.004524886877828054
  }
}
{'max_iter': 506, 'penalty': 'l2', 'C': 0.0001545790425808255, 'l1_ratio': 4.7564929956506116e-08}


In [20]:
##                          ##
## RANDOM FOREST CLASSIFIER ##
##                          ##
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()

param_grid = {
    'n_estimators': Integer(10, 100),
    'max_depth':Integer(3, 10),
    'max_leaf_nodes':Integer(1, 100),
    'max_features':Categorical(['sqrt', 'log2', None]),
    'criterion':Categorical(['gini', 'entropy']),
    'min_weight_fraction_leaf':Continuous(0.01, 0.5, distribution='log-uniform')
}

cv = StratifiedKFold(n_splits=3, shuffle=True)

evolved_estimator = GASearchCV(estimator=rf, cv=cv, scoring='accuracy',
                               population_size=10, generations=10,
                               tournament_size=3, elitism=True,
                               crossover_probability=0.8, mutation_probability=0.1,
                               param_grid=param_grid, criteria='max',
                               algorithm='eaMuPlusLambda', n_jobs=-1,
                               verbose=True, keep_top_k=4)

In [21]:
_start =time.time()

evolved_estimator.fit(X_train, y_train)

_end = time.time()

# only in seconds

print(f'running for: {int((_end-_start)/60)}m, {int((_end-_start)%60)}s')

gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	10    	0.836963	0          	0.836963   	0.836963   
1  	18    	0.836963	0          	0.836963   	0.836963   
2  	20    	0.836963	0          	0.836963   	0.836963   
3  	16    	0.836963	0          	0.836963   	0.836963   
4  	16    	0.836963	0          	0.836963   	0.836963   
5  	18    	0.836963	0          	0.836963   	0.836963   
6  	17    	0.836963	0          	0.836963   	0.836963   
7  	19    	0.836963	0          	0.836963   	0.836963   
8  	15    	0.836963	0          	0.836963   	0.836963   
9  	16    	0.836963	0          	0.836963   	0.836963   
10 	16    	0.836963	0          	0.836963   	0.836963   
running for: 1m, 33s


In [22]:
print(json.dumps(analyse(evolved_estimator, X_train, y_train, X_test, y_test), indent=2))

print(evolved_estimator.best_params_)

{
  "train": {
    "pr auc": 0.2691579827113811,
    "accuracy": 0.8369630071599046,
    "recall": 0.0,
    "precision": 0.0,
    "roc auc": 0.6644491451978967,
    "f1 score": 0.0
  },
  "test": {
    "pr auc": 0.24968613942036127,
    "accuracy": 0.8469032707028532,
    "recall": 0.0,
    "precision": 0.0,
    "roc auc": 0.6575642414282513,
    "f1 score": 0.0
  }
}
{'n_estimators': 26, 'max_depth': 8, 'max_leaf_nodes': 32, 'max_features': 'log2', 'criterion': 'entropy', 'min_weight_fraction_leaf': 0.1804810940579604}


In [23]:
##                     ##
## XG BOOST CLASSIFIER ##
##                     ##

from xgboost.sklearn import XGBClassifier

param_grid = {
    'n_estimators': Integer(10, 20),
    'max_depth': Integer(3, 9),
    'gamma': Continuous(1e-6, 1, distribution='log-uniform'),
    'learning_rate': Continuous(1e-6, 1e-2, distribution='log-uniform')
}


xgb = XGBClassifier();

cv = StratifiedKFold(n_splits=3, shuffle=True)

evolved_estimator = GASearchCV(estimator=xgb, cv=cv, scoring='accuracy',
                               population_size=10, generations=10,
                               tournament_size=3, elitism=True,
                               crossover_probability=0.8, mutation_probability=0.1,
                               param_grid=param_grid, criteria='max',
                               algorithm='eaMuPlusLambda', n_jobs=-1,
                               verbose=True, keep_top_k=4)

In [24]:
_start =time.time()

evolved_estimator.fit(X_train, y_train)

_end = time.time()

# only in seconds

print(f'running for: {int((_end-_start)/60)}m, {int((_end-_start)%60)}s')

gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	10    	0.824597	0.00577502 	0.834577   	0.816676   
1  	17    	0.832593	0.00434928 	0.838306   	0.823986   
2  	20    	0.833428	0.00453549 	0.837411   	0.823986   
3  	18    	0.837023	0.000650967	0.837411   	0.835769   
4  	18    	0.836918	0.000663946	0.837411   	0.83577    
5  	19    	0.837381	8.95258e-05	0.837411   	0.837112   
6  	18    	0.837202	0.00063699 	0.83756    	0.835322   
7  	18    	0.837142	0.000338631	0.83741    	0.836516   
8  	17    	0.83741 	0.000346593	0.838007   	0.836665   
9  	19    	0.837411	0.000663891	0.838007   	0.83562    
10 	16    	0.83671 	0.000822527	0.838007   	0.83577    
running for: 0m, 47s


In [25]:
print(json.dumps(analyse(evolved_estimator, X_train, y_train, X_test, y_test), indent=2))

print(evolved_estimator.best_params_)

{
  "train": {
    "pr auc": 0.25168195490040907,
    "accuracy": 0.8386038186157518,
    "recall": 0.0384263494967978,
    "precision": 0.5753424657534246,
    "roc auc": 0.6477793342478659,
    "f1 score": 0.07204116638078902
  },
  "test": {
    "pr auc": 0.23028574834066018,
    "accuracy": 0.8427279053583855,
    "recall": 0.020454545454545454,
    "precision": 0.3,
    "roc auc": 0.6402241913796968,
    "f1 score": 0.03829787234042553
  }
}
{'n_estimators': 20, 'max_depth': 3, 'gamma': 0.00011636742716756199, 'learning_rate': 1.2158712076584003e-06}


In [26]:
##                        ##
## NAIVE BAYES CLASSIFIER ##
##                        ##


from sklearn.naive_bayes import GaussianNB
import json
param_grid = {
    'var_smoothing': Continuous(1e-9, 1, distribution='log-uniform')
}

gnb = GaussianNB();

cv = StratifiedKFold(n_splits=3, shuffle=True)

evolved_estimator = GASearchCV(estimator=gnb, cv=cv, scoring='accuracy',
                               population_size=10, generations=10,
                               tournament_size=3, elitism=True,
                               crossover_probability=0.8, mutation_probability=0.1,
                               param_grid=param_grid, criteria='max',
                               algorithm='eaMuPlusLambda', n_jobs=-1,
                               verbose=True, keep_top_k=4)

In [27]:
_start =time.time()

evolved_estimator.fit(X_train, y_train)

_end = time.time()

# only in seconds

print(f'running for: {int((_end-_start)/60)}m, {int((_end-_start)%60)}s')

gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	10    	0.811933	0.0288724  	0.837112   	0.763575   
1  	17    	0.836351	0.000807207	0.837112   	0.834875   
2  	19    	0.836978	0.000123852	0.837112   	0.836665   
3  	19    	0.836993	5.9657e-05 	0.837112   	0.836963   
4  	18    	0.837023	9.8946e-05 	0.837261   	0.836963   
5  	19    	0.837082	0.000130047	0.837261   	0.836963   
6  	17    	0.837142	0.000111633	0.837261   	0.836963   
7  	16    	0.837112	0.000115534	0.837261   	0.836963   
8  	19    	0.837127	0.000140699	0.837261   	0.836963   
9  	20    	0.837067	0.000221752	0.837261   	0.836515   
10 	19    	0.837067	0.000116471	0.837261   	0.836963   
running for: 0m, 10s


In [28]:
print(json.dumps(analyse(evolved_estimator, X_train, y_train, X_test, y_test), indent=2))

print(evolved_estimator.best_params_)

{
  "train": {
    "pr auc": 0.22699092075141608,
    "accuracy": 0.8371121718377088,
    "recall": 0.0018298261665141812,
    "precision": 0.6666666666666666,
    "roc auc": 0.6191607029911022,
    "f1 score": 0.0036496350364963507
  },
  "test": {
    "pr auc": 0.21980718372185432,
    "accuracy": 0.8455114822546973,
    "recall": 0.0022727272727272726,
    "precision": 0.16666666666666666,
    "roc auc": 0.622773026070068,
    "f1 score": 0.004484304932735426
  }
}
{'var_smoothing': 0.0015023982889710696}


In [29]:
##                ##
## SVC CLASSIFIER ##
##                ##


from sklearn.svm import SVC

param_grid = {
    'C': Continuous(1e-4, 10, distribution='log-uniform'),
    'gamma': Continuous(1e-4, 10, distribution='log-uniform')
}

svc = SVC()

cv = StratifiedKFold(n_splits=3, shuffle=True)

evolved_estimator = GASearchCV(estimator=svc, cv=cv, scoring='accuracy',
                               population_size=10, generations=10,
                               tournament_size=3, elitism=True,
                               crossover_probability=0.8, mutation_probability=0.1,
                               param_grid=param_grid, criteria='max',
                               algorithm='eaMuPlusLambda', n_jobs=-1,
                               verbose=True, keep_top_k=4)

In [30]:
_start =time.time()

evolved_estimator.fit(X_train, y_train)

_end = time.time()

# only in seconds

print(f'running for: {int((_end-_start)/60)}m, {int((_end-_start)%60)}s')

gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	10    	0.836963	0          	0.836963   	0.836963   
1  	20    	0.836963	0          	0.836963   	0.836963   
2  	18    	0.836963	0          	0.836963   	0.836963   
3  	18    	0.836963	0          	0.836963   	0.836963   
4  	16    	0.836963	0          	0.836963   	0.836963   
5  	17    	0.836963	0          	0.836963   	0.836963   
6  	18    	0.836963	0          	0.836963   	0.836963   
7  	19    	0.836963	0          	0.836963   	0.836963   
8  	18    	0.836963	0          	0.836963   	0.836963   
9  	18    	0.836963	0          	0.836963   	0.836963   
10 	19    	0.836963	0          	0.836963   	0.836963   
running for: 7m, 5s


In [31]:
print(json.dumps(analyse(evolved_estimator, X_train, y_train, X_test, y_test), indent=2))

print(evolved_estimator.best_params_)

{
  "train": {
    "pr auc": 0.7108014555230501,
    "accuracy": 0.8369630071599046,
    "recall": 0.0,
    "precision": 0.0,
    "roc auc": 0.8647316252238162,
    "f1 score": 0.0
  },
  "test": {
    "pr auc": 0.17393034121511802,
    "accuracy": 0.8469032707028532,
    "recall": 0.0,
    "precision": 0.0,
    "roc auc": 0.5212659669828938,
    "f1 score": 0.0
  }
}
{'C': 0.0011342078116549125, 'gamma': 0.003095780557248269}


In [32]:
##                ##
## ANN CLASSIFIER ##
##                ##

import random

class foo(Categorical):
    def __init__(self):
        pass
    def sample(self):
        hidden_layer_count = random.randint(1, 4)
        l = random.sample(range(8, 32), hidden_layer_count)
        return tuple(l)

param_grid = {
    'alpha': Continuous(1e-4, 0.05, distribution='log-uniform'),
    'activation': Categorical(['tanh', 'relu', 'identity', 'logistic']),
    'solver': Categorical(['lbfgs', 'sgd', 'adam']),
    'batch_size': Integer(100, 1000),
    'learning_rate': Categorical(['constant', 'invscaling', 'adaptive']),
    'hidden_layer_sizes': foo(),
    'max_iter': Integer(50, 150),
}

ann = Model()

cv = StratifiedKFold(n_splits=3, shuffle=True)

evolved_estimator = GASearchCV(estimator=ann, cv=cv, scoring='accuracy',
                               population_size=10, generations=10,
                               tournament_size=3, elitism=True,
                               crossover_probability=0.8, mutation_probability=0.1,
                               param_grid=param_grid, criteria='max',
                               algorithm='eaMuPlusLambda', n_jobs=-1,
                               verbose=True, keep_top_k=4)

In [33]:
_start =time.time()

evolved_estimator.fit(X_train, y_train)

_end = time.time()

# only in seconds

print(f'running for: {int((_end-_start)/60)}m, {int((_end-_start)%60)}s')

gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	10    	0.829849	0.0165912  	0.837858   	0.780736   
1  	18    	0.837082	0.00039924 	0.837858   	0.836665   
2  	19    	0.836963	0.000176558	0.837411   	0.836665   
3  	17    	0.837008	0.000134288	0.837411   	0.836963   
4  	17    	0.837053	0.000179051	0.837411   	0.836963   
5  	20    	0.837097	0.000205129	0.837411   	0.836963   
6  	18    	0.837053	0.000179051	0.837411   	0.836963   
7  	16    	0.837142	0.000219292	0.837411   	0.836963   
8  	20    	0.837112	0.000200185	0.837411   	0.836963   
9  	19    	0.837187	0.000191676	0.837411   	0.836963   
10 	19    	0.837008	0.00032032 	0.837411   	0.836516   
running for: 5m, 8s


In [34]:
import json
print(json.dumps(analyseANN(evolved_estimator, X_train, y_train, X_test, y_test), indent=2))

print(evolved_estimator.best_params_)

{
  "train": {
    "pr auc": 0.3117115683927123,
    "accuracy": 0.8363663484486874,
    "recall": 0.01555352241537054,
    "precision": 0.4473684210526316,
    "roc auc": 0.5059054370230479,
    "f1 score": 0.030061892130857647
  },
  "test": {
    "pr auc": 0.2587932933659325,
    "accuracy": 0.8451635351426583,
    "recall": 0.013636363636363636,
    "precision": 0.35294117647058826,
    "roc auc": 0.5045585269291103,
    "f1 score": 0.026258205689277902
  }
}
{'alpha': 0.0007070510425864872, 'activation': 'identity', 'solver': 'adam', 'batch_size': 552, 'learning_rate': 'constant', 'hidden_layer_sizes': (22, 30), 'max_iter': 76}
